In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("data_w_cluster.csv")
df.head()

,AGE,장르명,스타배우,할인율,예매/취소방식명(관리시스템),공연지역명_경기도,공연지역명_경상도,공연지역명_서울,공연지역명_전라도,공연지역명_제주도,...,계층_조합_연극_0,계층_조합_연극_1,계층_조합_연극_2,성별_1,성별_2,cluster,공연코드,재관람,공연일시,AGE_restored
0,-0.343027,1,0,0.0,0,False,False,False,False,True,...,True,False,False,False,True,2,PF304959,0,2022-06-02 14:00,33.0
1,-0.999494,0,0,0.0,1,False,False,True,False,False,...,False,False,False,False,False,1,PF300599,0,2022-06-09 20:00,26.0
2,-0.624370,0,1,0.0,0,False,False,True,False,False,...,False,False,False,False,True,3,PF308077,0,2022-06-09 19:30,30.0
3,0.501001,1,0,0.0,0,False,False,True,False,False,...,True,False,False,False,True,2,PF312815,0,2022-06-09 19:30,42.0
4,-0.718151,0,0,0.3,1,False,False,True,False,False,...,False,False,False,False,True,1,PF308025,0,2022-06-04 19:00,29.0


In [ ]:
df['재관람'].value_counts()

재관람
0    6724781
1      19078
Name: count, dtype: int64

클러스터 1 페르소나: {'연령': 51.07, '재관람': 0.003, '스타배우출연여부': 0.42, '할인율': 0.035, '예매/취소방식명': 0, '공연지역': '서울', '계층 조합': '뮤지컬_2', '성별': '성별 2'}

클러스터 2 페르소나: {'연령': 31.508, '재관람': 0.003, '스타배우출연여부': 0.17, '할인율': 0.064, '예매/취소방식명': 0, '공연지역': '서울', '계층 조합': '뮤지컬_1', '성별': '성별 2'}

클러스터 3 페르소나: {'연령': 31.74, '재관람': 0.002, '스타배우출연여부': 0.1, '할인율': 0.045, '예매/취소방식명': 0, '공연지역': '서울', '계층 조합': '연극_1', '성별': '성별 2'}

클러스터 4 페르소나: {'연령': 31.899, '재관람': 0.003, '스타배우출연여부': 0.79, '할인율': 0.017, '예매/취소방식명': 0, '공연지역': '서울', '계층 조합': '뮤지컬_2', '성별': '성별 2'}

In [ ]:
# 전체 재관람 비율
expected_revisit_rate = 19078 / (6724781+19078)

# 각 클러스터에 대한 재관람 비율을 정의합니다.
revisit_rates = {0: 0.003, 1: 0.003, 2: 0.002, 3: 0.003}

# 각 클러스터에 대해 'actual_revisit_rate' 컬럼 값을 업데이트합니다.
for cluster, rate in revisit_rates.items():
    df.loc[df['cluster'] == cluster, 'actual_revisit_rate'] = rate

# 기대 비율 대비 실제 비율의 차이 계산 (revisit_bias)
df['revisit_bias'] = df['actual_revisit_rate'] - expected_revisit_rate

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743859 entries, 0 to 6743858
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   AGE                  float64
 1   장르명                  int64  
 2   스타배우                 int64  
 3   할인율                  float64
 4   예매/취소방식명(관리시스템)      int64  
 5   공연지역명_경기도            bool   
 6   공연지역명_경상도            bool   
 7   공연지역명_서울             bool   
 8   공연지역명_전라도            bool   
 9   공연지역명_제주도            bool   
 10  공연지역명_충청도            bool   
 11  계층_조합_뮤지컬_1          bool   
 12  계층_조합_뮤지컬_2          bool   
 13  계층_조합_연극_0           bool   
 14  계층_조합_연극_1           bool   
 15  계층_조합_연극_2           bool   
 16  성별_1                 bool   
 17  성별_2                 bool   
 18  cluster              int64  
 19  공연코드                 object 
 20  재관람                  int64  
 21  공연일시                 object 
 22  AGE_restored         float64
 23  actual_revisit_rate  float64
 24

필요없는 컬럼 : 공연코드, 공연일시, AGE, 성별_1, 성별_2, 예매/취소방식명(관리시스템), AGE_restored, actual_revisit_rate

In [ ]:
df_pred = df.drop(columns = ['공연코드', '공연일시', 'AGE', '성별_1', '성별_2', '예매/취소방식명(관리시스템)', 'AGE_restored', 'actual_revisit_rate'])

In [ ]:
df_pred.head()

,장르명,스타배우,할인율,공연지역명_경기도,공연지역명_경상도,공연지역명_서울,공연지역명_전라도,공연지역명_제주도,공연지역명_충청도,계층_조합_뮤지컬_1,계층_조합_뮤지컬_2,계층_조합_연극_0,계층_조합_연극_1,계층_조합_연극_2,cluster,재관람,revisit_bias
0,1,0,0.0,False,False,False,False,True,False,False,False,True,False,False,2,0,-0.000829
1,0,0,0.0,False,False,True,False,False,False,True,False,False,False,False,1,0,0.000171
2,0,1,0.0,False,False,True,False,False,False,False,True,False,False,False,3,0,0.000171
3,1,0,0.0,False,False,True,False,False,False,False,False,True,False,False,2,0,-0.000829
4,0,0,0.3,False,False,True,False,False,False,True,False,False,False,False,1,0,0.000171


In [ ]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743859 entries, 0 to 6743858
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   장르명           int64  
 1   스타배우          int64  
 2   할인율           float64
 3   공연지역명_경기도     bool   
 4   공연지역명_경상도     bool   
 5   공연지역명_서울      bool   
 6   공연지역명_전라도     bool   
 7   공연지역명_제주도     bool   
 8   공연지역명_충청도     bool   
 9   계층_조합_뮤지컬_1   bool   
 10  계층_조합_뮤지컬_2   bool   
 11  계층_조합_연극_0    bool   
 12  계층_조합_연극_1    bool   
 13  계층_조합_연극_2    bool   
 14  cluster       int64  
 15  재관람           int64  
 16  revisit_bias  float64
dtypes: bool(11), float64(2), int64(4)
memory usage: 379.5 MB


In [ ]:
# revisit_bias 컬럼을 float32로 변환
df_pred['revisit_bias'] = df_pred['revisit_bias'].astype('float32')
df_pred['할인율'] = df_pred['할인율'].astype('float32')
# int64를 int32로 변환
df_pred = df_pred.astype({col: 'int32' for col in df_pred.select_dtypes(['int64']).columns})
# bool 타입을 int32로 변환
df_pred = df_pred.astype({col: 'int32' for col in df_pred.select_dtypes(['bool']).columns})

In [ ]:
# 클러스터별로 데이터를 분리
clusters = df_pred['cluster'].unique()

models = {}

# DNN 모델

In [ ]:
conda init

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
modified      /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
no change     /Users/lucianapark/.bash_profile

==> For changes to take effect, close and re-open your current shell. <==


Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install -c apple tensorflow-deps

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::_anaconda_depends==2024.02=py311_openblas_1
  - defaults/osx-arm64::tensorflow==2.12.0=eigen_py311h689d69b_0
unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: | ^C
unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install tensorflow

Solving environment: / ^C
unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import tensorflow as tf

# 모든 장치 목록을 출력합니다.
devices = tf.config.list_physical_devices('GPU')
if devices:
    print("GPU 장치가 사용 가능합니다:")
    for device in devices:
        print(device)
else:
    print("GPU 장치가 사용되지 않습니다.")

GPU 장치가 사용 가능합니다:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
# 각 클러스터에 대해 DNN 모델을 학습
for cluster in clusters:
    cluster_data = df_pred[df_pred['cluster'] == cluster]

    X = cluster_data.drop(['cluster', '재관람', 'revisit_bias'], axis=1)
    y = cluster_data['재관람']

    # DNN 모델 정의
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 모델 학습
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # 학습된 모델 저장
    models[cluster] = model

Epoch 1/10


2024-08-30 06:52:19.678593: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-08-30 06:52:19.678612: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-08-30 06:52:19.678615: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-08-30 06:52:19.678768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-30 06:52:19.678776: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-08-30 06:52:20.051495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


50619/50619 ━━━━━━━━━━━━━━━━━━━━ 240s 5ms/step - accuracy: 0.9981 - loss: 0.0106
Epoch 2/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 243s 5ms/step - accuracy: 0.9981 - loss: 0.0062
Epoch 3/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 243s 5ms/step - accuracy: 0.9982 - loss: 0.0060
Epoch 4/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 243s 5ms/step - accuracy: 0.9982 - loss: 0.0058
Epoch 5/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 241s 5ms/step - accuracy: 0.9982 - loss: 0.0058
Epoch 6/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 245s 5ms/step - accuracy: 0.9983 - loss: 0.0056
Epoch 7/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 242s 5ms/step - accuracy: 0.9983 - loss: 0.0053
Epoch 8/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 241s 5ms/step - accuracy: 0.9983 - loss: 0.0053
Epoch 9/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 242s 5ms/step - accuracy: 0.9985 - loss: 0.0053
Epoch 10/10
50619/50619 ━━━━━━━━━━━━━━━━━━━━ 244s 5ms/step - accuracy: 0.9986 - loss: 0.0050
Epoch 1/10
52142/52142 ━━━━━━━━━━━━━━━━━━━━ 252s 5ms/step - accuracy: 0.9966 - loss: 0.017

# 확률을 예측할 새로운 데이터(뮤_2)

공연 추출은 각각 분야별로 회차가 20회 이상인 것 중에서 티켓 수가 가장 많은 공연 3개를 뽑았다.

In [ ]:
#공연코드 'PF422052'
df_m2_1 = pd.DataFrame({
    '장르명': [0],
    '스타배우': [1],
    '할인율': [0.00],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [0],
    '계층_조합_뮤지컬_2': [1],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [0],
    '계층_조합_연극_2': [0],
})

In [ ]:
#공연코드 'PF316185'
df_m2_2 = pd.DataFrame({
    '장르명': [0],
    '스타배우': [0],
    '할인율': [0.009],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [0],
    '계층_조합_뮤지컬_2': [1],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [0],
    '계층_조합_연극_2': [0],
})

In [ ]:
#공연코드 'PF430694'
df_m2_3 = pd.DataFrame({
    '장르명': [0],
    '스타배우': [1],
    '할인율': [0.005],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [0],
    '계층_조합_뮤지컬_2': [1],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [0],
    '계층_조합_연극_2': [0],
})

# 확률예측(뮤_2)

In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_m2_1)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000465
클러스터 1: 재관람 확률 = 0.00000000
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_m2_2)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.03288861
클러스터 1: 재관람 확률 = 0.00008412
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000080


In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_m2_3)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000572
클러스터 1: 재관람 확률 = 0.00000000
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


# 확률을 예측할 새로운 데이터(뮤_1)

In [ ]:
#공연코드 PF435391
df_m1_1 = pd.DataFrame({
    '장르명': [0],
    '스타배우': [1],
    '할인율': [0.001],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [1],
    '계층_조합_뮤지컬_2': [0],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [0],
    '계층_조합_연극_2': [0],
})

In [ ]:
#공연코드 'PF311420'
df_m1_2 = pd.DataFrame({
    '장르명': [0],
    '스타배우': [1],
    '할인율': [0.000],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [1],
    '계층_조합_뮤지컬_2': [0],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [0],
    '계층_조합_연극_2': [0],
})

In [ ]:
#공연코드 'PF438620'
df_m1_3 = pd.DataFrame({
    '장르명': [0],
    '스타배우': [0],
    '할인율': [0.000],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [1],
    '계층_조합_뮤지컬_2': [0],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [0],
    '계층_조합_연극_2': [0],
})

# 확률예측(뮤_1)

In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_m1_1)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000027
클러스터 1: 재관람 확률 = 0.00000000
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_m1_2)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000026
클러스터 1: 재관람 확률 = 0.00000000
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_m1_3)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00216384
클러스터 1: 재관람 확률 = 0.00000000
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


# 확률을 예측할 새로운 데이터(연_1)

In [ ]:
#공연코드'PF352013'
df_p1_1 = pd.DataFrame({
    '장르명': [1],
    '스타배우': [0],
    '할인율': [0.011],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [0],
    '계층_조합_뮤지컬_2': [0],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [1],
    '계층_조합_연극_2': [0],
})

In [ ]:
#공연코드 'PF367079'
df_p1_2 = pd.DataFrame({
    '장르명': [1],
    '스타배우': [0],
    '할인율': [0.000],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [0],
    '계층_조합_뮤지컬_2': [0],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [1],
    '계층_조합_연극_2': [0],
})

In [ ]:
#공연코드'PF350515'
df_p1_3 = pd.DataFrame({
    '장르명': [1],
    '스타배우': [0],
    '할인율': [0.001],
    '공연지역명_경기도': [0],
    '공연지역명_경상도': [0],
    '공연지역명_서울': [1],
    '공연지역명_전라도': [0],
    '공연지역명_제주도': [0],
    '공연지역명_충청도': [0],
    '계층_조합_뮤지컬_1': [0],
    '계층_조합_뮤지컬_2': [0],
    '계층_조합_연극_0': [0],
    '계층_조합_연극_1': [1],
    '계층_조합_연극_2': [0],
})

# 확률예측(연_1)

In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_p1_1)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000000
클러스터 1: 재관람 확률 = 0.00000039
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_p1_2)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000000
클러스터 1: 재관람 확률 = 0.00000000
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000


In [ ]:
# 각 클러스터에 대해 재관람 확률 예측
predictions = {}

for cluster in clusters:
    # 모델로 예측
    prediction = models[cluster].predict(df_p1_3)[0][0]

    # revisit_bias를 가중치로 적용하여 확률 조정
    bias = df_pred[df_pred['cluster'] == cluster]['revisit_bias'].mean()
    adjusted_prediction = prediction * (3 + bias)

    predictions[cluster] = adjusted_prediction

# 결과 출력
print("클러스터별 재관람 확률 예측 (가중치 적용):")
for cluster, probability in predictions.items():
    print(f"클러스터 {cluster}: 재관람 확률 = {probability:.8f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
클러스터별 재관람 확률 예측 (가중치 적용):
클러스터 2: 재관람 확률 = 0.00000000
클러스터 1: 재관람 확률 = 0.00000001
클러스터 3: 재관람 확률 = 0.00000000
클러스터 0: 재관람 확률 = 0.00000000
